Basic Editor
  User needs to set userkey, and database host if necessary.
  
  Can be used to select and edit more detailed properties of threat events
     - run first two cells, and then either select an event to edit or add a new event
     - run the next cell to load the selected event
     - save the event using the "save threat event" button, 
           using the notebook save button will not save the event's data

In [ ]:
userkey = "140"
dbhost  = 'localhost'

In [ ]:
import ipywidgets as widgets
import psycopg2
import qual

conn = psycopg2.connect("postgres://csc452:csc452@" + dbhost + "/csc452")  
cursor = conn.cursor()  
cursor.execute('SET mydomain.userkey = ' + userkey)


num_range = range(0,100) 

add_button = widgets.Button(description="add threat event")
def do_add(b):
    cursor.execute (qual.new_triage_stmt, ["new threat", "threat description here", 50, 50, 50, "impact name", "impact desc", "source name", "source desc", 50, 50])
    conn.commit()

add_button.on_click(do_add)

cursor.execute ('SELECT threat_id, source_id, impact_id, name, description, vulnerability, pervasive, impact, impact_name, impact_description, source_name, source_description, capacity, init FROM triagev')
triages = cursor.fetchall()

rec = {}

def update_row():
    print ("saving threat event ")
    row = rec['row']
    w = rec['w']
    vuln_val = w[5].value
    perv_val = w[6].value
    imp_val = w[7].value
    cap_val = w[12].value
    init_val = w[13].value
    cursor.execute (qual.update_triage_stmt, [w[3].value, w[4].value, vuln_val, perv_val, imp_val, w[8].value, w[9].value, w[10].value, w[11].value, cap_val, init_val, row[0], row[1], row[2]])
    conn.commit()
    
display (add_button)

# allow user to select an event to edit

# build up a list of the available events
options_list = []
options_mapping = {}

for x in range(0, len(triages)):
    options_list.append (triages[x][3])
    options_mapping[triages[x][3]] = x

# now create a widget
select_widget = widgets.Dropdown (options=options_list, value=options_list[0], description='Threat Events:')
display (select_widget)


Use the above selection to choose an event to edit, and then run the cell below to pull up that event data.

In [ ]:
x = options_mapping[select_widget.value]

w = [0,0,0]
w.append ( widgets.Text( value=triages[x][3], description='Threat Event:' ))
w.append ( widgets.Textarea( value=triages[x][4], description='Description:' ))
w.append ( widgets.Dropdown( options=num_range, value=qual.norm(triages[x][5]), description='Vulnerability' ))
w.append ( widgets.Dropdown( options=num_range, value=qual.norm(triages[x][6]), description='Pervasiveness' ))
w.append ( widgets.Dropdown( options=num_range, value=qual.norm(triages[x][7]), description='Impact' ))
w.append ( widgets.Text( value=triages[x][8], description='Impact:' ))
w.append ( widgets.Textarea( value=triages[x][9], description='Impact Description:' ))
w.append ( widgets.Text( value=triages[x][10], description='Threat Source:' ))
w.append ( widgets.Textarea( value=triages[x][11], description='Source Description:' ))
w.append ( widgets.Dropdown( options=num_range, value=qual.norm(triages[x][12]), description='Threat Capacity:' ))
w.append ( widgets.Dropdown( options=num_range, value=qual.norm(triages[x][13]), description='Attack Likeliness:' ))
rec['row'] = triages[x]
rec['w'] = w
b = widgets.Button (description = 'save threat event')
    
    
def make_fn ():
    return lambda b: update_row()
         
b.on_click(make_fn())

print ("After a save, to refresh rerun both this cell and the one above.")
    
display (widgets.VBox ([
        widgets.HBox([w[3], w[4]]),
        widgets.HBox([w[5], w[6]]),
        w[7],
        widgets.HBox([w[8], w[9]]),
        widgets.HBox([w[10], w[11]]),
        widgets.HBox([w[12], w[13]]),
        b,
        widgets.Label ()
    ]))